In [19]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

df = pd.read_csv(r"D:\Car Sales Analysis\Car Sales.xlsx - car_data.csv")

df = df.dropna()

def clean_names(series):
    return series.str.lower().str.strip().str.replace(" ", "", regex=False)

print("Unique companies before cleaning:", df['Company'].unique())

df['Company_cleaned'] = clean_names(df['Company'])
df['Model_cleaned'] = clean_names(df['Model'])

print("Unique companies after cleaning:", df['Company_cleaned'].unique())

ford_entries = df[df['Company_cleaned'] == 'ford']
print("Entries for 'Ford' after cleaning:")
print(ford_entries)

label_encoders = {}
categorical_features = ['Body Style', 'Engine']

for feature in categorical_features:
    label_encoders[feature] = LabelEncoder()
    df[feature] = label_encoders[feature].fit_transform(df[feature])

df['Sales Volume'] = df.groupby(['Dealer_Name', 'Model_cleaned'])['Model_cleaned'].transform('count')

df['Revenue'] = df['Price ($)'] * df['Sales Volume']

profit_margin = 0.20
df['Estimated Profit'] = df['Revenue'] * profit_margin

model_profit_analysis = df.groupby('Model_cleaned').agg({
    'Price ($)': 'mean',
    'Sales Volume': 'sum',
    'Revenue': 'sum',
    'Estimated Profit': 'sum'
}).reset_index()

print("\nModel Profit Analysis:")
print(model_profit_analysis)

def price_recommendation_for_model(company_name, model_name, df, profit_margin=0.20):
    company_name_cleaned = clean_names(pd.Series([company_name])).values[0]
    model_name_cleaned = clean_names(pd.Series([model_name])).values[0]

    print(f"Looking for company: '{company_name_cleaned}'")
    print(f"Looking for model: '{model_name_cleaned}'")
    
    company_data = df[df['Company_cleaned'] == company_name_cleaned]
    
    if company_data.empty:
        print(f"No data found for company '{company_name_cleaned}' after cleaning.")
        return
    
    revenue_data = company_data.groupby('Model_cleaned').agg(
        Price=('Price ($)', 'mean'),  # Average price
        Sales_Volume=('Model_cleaned', 'count'),  # Total sales count
        Revenue=('Revenue', 'sum')  # Total revenue for each model
    ).reset_index()

    revenue_data['Estimated Profit'] = revenue_data['Revenue'] * profit_margin  # Total estimated profit for each model

    revenue_data = revenue_data[revenue_data['Sales_Volume'] > 1]

    if revenue_data.empty:
        print("No models found with sufficient sales volume.")
        return

    print("Revenue and Profit data for models in company:")
    print(revenue_data)

    model_data = revenue_data[revenue_data['Model_cleaned'] == model_name_cleaned]
    if not model_data.empty:
        X = revenue_data['Sales_Volume'].values.reshape(-1, 1)
        y = revenue_data['Price'].values
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
        rf_model.fit(X_train, y_train)
        
        sales_volume = model_data['Sales_Volume'].values[0]  # Sales volume of the specific model
        optimal_price = rf_model.predict([[sales_volume]])[0]
        
        estimated_revenue = optimal_price * sales_volume
        estimated_profit = estimated_revenue * profit_margin

        print(f"Recommended price for model '{model_name_cleaned}': ${optimal_price:.2f} (Sales Volume: {sales_volume})")
        print(f"Estimated profit for model '{model_name_cleaned}': ${estimated_profit:.2f} (Revenue: ${estimated_revenue:.2f})")
    else:
        print(f"No sales data found for the model '{model_name_cleaned}'.")

    recommended_model = revenue_data.loc[revenue_data['Estimated Profit'].idxmax()]

    

    average_price = 23682.84 
    sales_volume = 121  

    revenue_average = average_price * sales_volume
    estimated_profit_average = revenue_average * profit_margin

    revenue_recommended = optimal_price * sales_volume
    estimated_profit_recommended = revenue_recommended * profit_margin

    print("\n--- Profit and Revenue Comparison for 'Windstar' ---")
    print(f"Using average price of ${average_price}:")
    print(f"  - Revenue: ${revenue_average:.2f}")
    print(f"  - Estimated Profit: ${estimated_profit_average:.2f}")

    print(f"Using recommended price of ${optimal_price:.2f}:")
    print(f"  - Revenue: ${revenue_recommended:.2f}")
    print(f"  - Estimated Profit: ${estimated_profit_recommended:.2f}")

    profit_increase = estimated_profit_recommended - estimated_profit_average
    revenue_increase = revenue_recommended - revenue_average

    print(f"\nProfit increase with recommended price: ${profit_increase:.2f}")
    print(f"Revenue increase with recommended price: ${revenue_increase:.2f}")

price_recommendation_for_model('Ford', 'Windstar', df)


Unique companies before cleaning: ['Ford' 'Dodge' 'Cadillac' 'Toyota' 'Acura' 'Mitsubishi' 'Chevrolet'
 'Nissan' 'Mercury' 'BMW' 'Chrysler' 'Subaru' 'Hyundai' 'Honda' 'Infiniti'
 'Audi' 'Porsche' 'Volkswagen' 'Buick' 'Saturn' 'Mercedes-B' 'Jaguar'
 'Volvo' 'Pontiac' 'Lincoln' 'Oldsmobile' 'Lexus' 'Plymouth' 'Saab' 'Jeep']
Unique companies after cleaning: ['ford' 'dodge' 'cadillac' 'toyota' 'acura' 'mitsubishi' 'chevrolet'
 'nissan' 'mercury' 'bmw' 'chrysler' 'subaru' 'hyundai' 'honda' 'infiniti'
 'audi' 'porsche' 'volkswagen' 'buick' 'saturn' 'mercedes-b' 'jaguar'
 'volvo' 'pontiac' 'lincoln' 'oldsmobile' 'lexus' 'plymouth' 'saab' 'jeep']
Entries for 'Ford' after cleaning:
             Car_id        Date Customer Name  Gender  Annual Income  \
0      C_CND_000001    1/2/2022     Geraldine    Male          13500   
9      C_CND_000010    1/2/2022       Grayson  Female          13500   
73     C_CND_000074    1/5/2022        Ingrid    Male         300000   
77     C_CND_000078    1/5/202